When I write '-----------' it means that I am about to write what is not going to go in the script, and when I write '________' it means that I stop writing what is not going to go in the script

In [8]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
'--------'
import traceback
'_________'



'_________'

In [4]:
browser = webdriver.Chrome() 

In [ ]:
page_number = 1
number_iteration=1
ids_cell_phones = []



'''I execute the infinite while loop that will stop when it no longer detects any more pages 
with products within the loop, mainly the identifier of each cell phone will be obtained and 
it will be stored in ids_cell_phones.'''

while True:

    url = f'https://tienda.claro.com.ar/plp/equipos?pageNumber={page_number}'

    browser.get(url)

    time.sleep(random.randint(8,10))


    '''I try to click on a cookie acceptance element if it is present.'''
    try:
        browser.find_element('xpath', '//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass
        
    html = browser.page_source

    soup = bs(html , 'lxml')

    final_page = int(soup.find('div',{'class':'Pagination_cont__28-ql'}).find_all('div',{'class': 'Pagination_number__3cAx4'})[-1].text)
    

    if page_number > final_page:        
        break

    page_number += 1


    articles = soup.find('div', {'class': 'Catalog_cont-fil-items__31RhA Catalog_filters-on__2NU5y'}).find('div', {'class': 'Catalog_items__3tiDK Catalog_grillax3__1AkL4'}).find_all('div', {'class': 'CardProducto_cont-sin-comparar__P90uJ'})

    for article in articles:
        cell_phone_id = article.find('a').get('href')

        ids_cell_phones.append(cell_phone_id)
      
        #print(article.find('a').get('href'))

ids_cell_phones = [phone for phone in ids_cell_phones if phone is not None]




In [12]:
print('Total number of cell phones:', len(ids_cell_phones))

Total number of cell phones: 50


In [ ]:


data = {
    'url/id': [],
    'error': []
}

critical_errors = pd.DataFrame(data)
errors_founds = pd.Series()

claro_cell_phones = pd.Series()

'''I define the function parce_cell_phones that takes a cell phone identifier 
as an argument and extracts detailed information about that phone from its individual page.'''

def parse_cell_phones(cell_phone_id):
    
    
    url = f'https://tienda.claro.com.ar{cell_phone_id}'
    #print('Direcciòn: ',url)

    try:
        browser.get(url)

        time.sleep(random.randint(5,12))

        html = browser.page_source

        soup = bs(html, 'lxml')

    
        model = soup.find('h1',{'data-test':'product_title'}).text

        price = int(soup.find('div', {'data-test': 'product_information'}).find('div',{'data-test':'product_prices'}).text.replace('$','').replace('.',''))


        especifications_1 = soup.find('div', {'class': 'Pdp_divFlexEspecificaciones__1ote3'}).find_all('div', {'class': 'Pdp_divIconFlex__3Hi6N'})
        especifications_2 = soup.find('div', {'class': 'Pdp_divTableEspecificaciones__3-bXX'}).find_all('div', {'class':'Pdp_divFlex__1AwU4'})

    except:
        '-----------'
        errors_founds['url/id'] = url
        errors_founds['error'] = traceback.format_exc()
        df_errors_founds = pd.DataFrame(errors_founds)
        global critical_errors
        critical_errors = pd.concat([critical_errors, df_errors_founds.T])
        '_________'
        return None


    ram = None
    camera = None
    processor = None
    storege = None
    screen = None

    for div in especifications_1:
        
            momentary_text = div.text.lower()

            if 'cámara' in momentary_text:
                try:
                    camera = ' / Trasera '+ div.find('div', {'class':'Pdp_descriptionImg__2wfms'}).text
                except:
                    pass
                    
            elif 'display' in momentary_text:
                try:
                    screen = div.find('div', {'class':'Pdp_descriptionImg__2wfms'}).text
                except:
                      pass
            
            elif 'procesador' in momentary_text:
                    try:
                        processor = div.find('div', {'class':'Pdp_descriptionImg__2wfms'}).text
                    except:
                        pass



    

    for div in especifications_2:
            momentary_text = div.text.lower()

            if 'cámara' in   momentary_text:
                try:
                    camera = 'Frontal ' + div.find('div',{'class':'Pdp_divDescripcion__36s2x'}).text + camera
                except:
                    pass

            elif 'ram' in momentary_text:
                try:
                    ram =  div.find('div',{'class':'Pdp_divDescripcion__36s2x'}).text
                except:
                    pass

            elif 'memoria interna' in momentary_text:
                try:
                    storege =  div.find('div',{'class':'Pdp_divDescripcion__36s2x'}).text
                except:
                    pass

    '-----------'
    if None in [ram, camera, processor, storege, screen]:
        errors = url
    else:
        errors = None
    '___________'

    claro_cell_phones['company'] = 'Claro'

    claro_cell_phones['model'] = model
    
    claro_cell_phones['price'] = price

    claro_cell_phones['ram'] = ram
    
    claro_cell_phones['processor'] = processor
    
    claro_cell_phones['storege'] = storege
    
    claro_cell_phones['camera'] = camera
    
    claro_cell_phones['screen'] = screen

    '--------'
    claro_cell_phones['errors'] = errors
    '________'

    df_claro_cell_phones = pd.DataFrame(claro_cell_phones)

    return (df_claro_cell_phones.T)


In [26]:
'''I create a for loop to iterate through the cell phone identifiers, call the pare_cell_phones function with 
each identifier, and add the results to the df_claro_cell_phones DataFrame.'''

df_claro_cell_phones = pd.DataFrame()

for i in range(len(ids_cell_phones)):
    try:
        df_claro_cell_phones = pd.concat([df_claro_cell_phones, parse_cell_phones(ids_cell_phones[i])])
    except:
        pass

    if len(critical_errors) > 10:
            print('Extraction stopped, to many errors')
            break

    print(f'Progress {i+1} of {len(ids_cell_phones)}')

    time.sleep(random.randint(4,8))

#browser.quit()

df_claro_cell_phones.to_csv('celulares_personal.csv', index = False)

print('Number of records obtained: ', len(df_claro_cell_phones))

print('Number of critical errors found: ', len(critical_errors))

print('Number of null data found: ',df_claro_cell_phones.iloc[:,:7].isnull().sum().sum())

print('extraction completed')

Progress 1 of 50
Progress 2 of 50
Progress 3 of 50
Progress 4 of 50
Progress 5 of 50
Progress 6 of 50
Progress 7 of 50
Progress 8 of 50
Progress 9 of 50
Progress 10 of 50
Progress 11 of 50
Progress 12 of 50
Progress 13 of 50
Progress 14 of 50
Progress 15 of 50
Progress 16 of 50
Progress 17 of 50
Progress 18 of 50
Progress 19 of 50
Progress 20 of 50
Progress 21 of 50
Progress 22 of 50
Progress 23 of 50
Progress 24 of 50
Progress 25 of 50
Progress 26 of 50
Progress 27 of 50
Progress 28 of 50
Progress 29 of 50
Progress 30 of 50
Progress 31 of 50
Progress 32 of 50
Progress 33 of 50
Progress 34 of 50
Progress 35 of 50
Progress 36 of 50
Progress 37 of 50
Progress 38 of 50
Progress 39 of 50
Progress 40 of 50
Progress 41 of 50
Progress 42 of 50
Progress 43 of 50
Progress 44 of 50
Progress 45 of 50
Progress 46 of 50
Progress 47 of 50
Progress 48 of 50
Progress 49 of 50
Progress 50 of 50
Number of records obtained:  49
Number of serious errors found:  1
Number of null data found:  0
extraction c

In [ ]:
df_claro_cell_phones

In [24]:
len(df_claro_cell_phones)

48

In [29]:
critical_errors['url/id'].iloc[0]

'https://tienda.claro.com.ar/pdp/equipos/samsung-galaxy-z-fold5-5g-256gb'

In [32]:
print(critical_errors['error'].iloc[0])

Traceback (most recent call last):
  File "/tmp/ipykernel_5808/301661863.py", line 32, in parse_cell_phones
    model = soup.find('h1',{'data-test':'product_title'}).text
AttributeError: 'NoneType' object has no attribute 'text'

